In [15]:
import numpy as np
import open3d as o3d
from scipy.spatial import cKDTree
import matplotlib

# --- Load data ---
nodes = np.load("output/predicted_points.npy")              # (N, 3)
edges = np.loadtxt("plate_surface_edges.txt", dtype=int, delimiter=",")
errors = np.load("output/error_magnitude.npy")              # (N,)

# --- Parameters ---
points_per_edge = 10   # how many points (excluding endpoints) to add per edge
new_points = []
new_errors = []

for i, j in edges:
    p1, p2 = nodes[i], nodes[j]
    e1, e2 = errors[i], errors[j]
    # interpolation parameters (excluding 0 and 1)
    t_values = np.linspace(0, 1, points_per_edge + 2)[1:-1]
    # interpolate positions
    edge_points = (1 - t_values[:, None]) * p1 + t_values[:, None] * p2
    new_points.append(edge_points)
    # interpolate error magnitudes
    edge_errors = (1 - t_values) * e1 + t_values * e2
    new_errors.append(edge_errors)

# --- Concatenate results ---
new_points = np.vstack(new_points)
new_errors = np.hstack(new_errors)

# --- Combine original + interpolated ---
nodes = np.vstack([nodes, new_points])
errors = np.hstack([errors, new_errors])

# --- Build Open3D point cloud ---
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(nodes)

# --- Estimate normals ---
print("Estimating normals...")
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=10))
#pcd.orient_normals_consistent_tangent_plane(10)
pcd.orient_normals_towards_camera_location(camera_location=np.mean(nodes,axis=0))

# --- Run Poisson reconstruction ---
print("Running Poisson surface reconstruction...")
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=15
    )

# --- Remove low-density vertices (optional cleanup) ---
#densities = np.asarray(densities)
#density_threshold = np.quantile(densities, 0.01)
#vertices_to_keep = densities > density_threshold
#mesh = mesh.select_by_index(np.where(vertices_to_keep)[0])

# --- Interpolate error values to mesh vertices ---
print("Interpolating error magnitudes onto mesh vertices...")
mesh_vertices = np.asarray(mesh.vertices)

# Build KDTree for nearest-neighbor lookup
tree = cKDTree(nodes)
dists, idxs = tree.query(mesh_vertices, k=1)
mesh_errors = errors[idxs]

# --- Normalize errors for color mapping ---
min_e, max_e = mesh_errors.min(), mesh_errors.max()
normed = (mesh_errors - min_e) / (max_e - min_e + 1e-8)

# --- Apply a colormap (e.g., 'viridis', 'plasma', 'jet') ---
cmap = matplotlib.colormaps.get_cmap('plasma')
colors = cmap(normed)[:, :3]  # drop alpha
mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

# --- Visualize mesh ---
print("Visualizing colored mesh...")
o3d.visualization.draw_geometries(
    [mesh],
    zoom=0.664,
    front=[-0.4761, -0.4698, -0.7434],
    lookat=[1.8900, 3.2596, 0.9284],
    up=[0.2304, -0.8825, 0.4101]
)



Estimating normals...
Running Poisson surface reconstruction...
[Open3D DEBUG] Input Points / Samples: 63506 / 63506
[Open3D DEBUG] #   Got kernel density: 1.2747011184692383 (s), 2927.859375 (MB) / 4900.02734375 (MB) / 4900 (MB)
[Open3D DEBUG] #     Got normal field: 0.21396398544311523 (s), 2964.109375 (MB) / 4900.02734375 (MB) / 4900 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 3.093643e-05 / 1.964649e+00
[Open3D DEBUG] #       Finalized tree: 1.8142378330230713 (s), 3403.484375 (MB) / 4900.02734375 (MB) / 4900 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.5236208438873291 (s), 2969.6171875 (MB) / 4900.02734375 (MB) / 4900 (MB)
[Open3D DEBUG] #Set point constraints: 0.2093369960784912 (s), 2969.6171875 (MB) / 4900.02734375 (MB) / 4900 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 15170744 / 1279656 / 16058337
[Open3D DEBUG] Memory Usage: 2969.617 MB
Cycle[0] Depth[ 0/20]:	Updated constraints / Got system / Solved in:  0.000 /  0.000 /  0.000	(3884.938 MB)	Nodes:

In [8]:
import numpy as np
import open3d as o3d
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib

# --- Load your data ---
nodes = np.load("output/predicted_points.npy")              # (N, 3)
errors = np.load("output/error_magnitude.npy")              # (N,)
edges = np.loadtxt("plate_surface_edges.txt", dtype=int, delimiter=",")

# If your edges are 1-based (FEM convention), convert to 0-based
if np.min(edges) == 1:
    edges -= 1

# --- Normalize errors to [0, 1] for colormap mapping ---
norm = mcolors.Normalize(vmin=np.min(errors), vmax=np.max(errors))
cmap = matplotlib.colormaps.get_cmap('viridis')
# --- Function to create a colored cylinder between two points ---
def make_edge_cylinder(p1, p2, color, radius=0.0015, resolution=16):
    """Create a cylinder mesh between points p1 and p2, with given RGB color."""
    # Cylinder aligned along z-axis by default
    height = np.linalg.norm(p2 - p1)
    cylinder = o3d.geometry.TriangleMesh.create_cylinder(radius=radius, height=height, resolution=resolution)
    cylinder.paint_uniform_color(color)

    # Align with edge direction
    direction = (p2 - p1) / height
    z_axis = np.array([0, 0, 1])
    if np.allclose(direction, z_axis):
        R = np.eye(3)
    elif np.allclose(direction, -z_axis):
        R = o3d.geometry.get_rotation_matrix_from_axis_angle(np.pi * np.array([1, 0, 0]))
    else:
        axis = np.cross(z_axis, direction)
        axis /= np.linalg.norm(axis)
        angle = np.arccos(np.dot(z_axis, direction))
        R = o3d.geometry.get_rotation_matrix_from_axis_angle(axis * angle)

    cylinder.rotate(R, center=(0, 0, 0))
    cylinder.translate(p1)
    return cylinder


# --- Build a list of colored cylinders (one per edge) ---
cylinder_meshes = []
for e in edges:
    p1, p2 = nodes[e[0]], nodes[e[1]]
    avg_err = (errors[e[0]] + errors[e[1]]) / 2
    color = cmap(norm(avg_err))[:3]
    cylinder_meshes.append(make_edge_cylinder(p1, p2, color, radius=0.0015))

# Combine all cylinders into one mesh for visualization
full_mesh = o3d.geometry.TriangleMesh()
for cyl in cylinder_meshes:
    full_mesh += cyl

# Optional: smooth shading
full_mesh.compute_vertex_normals()

# --- Visualize the thick wireframe ---
o3d.visualization.draw([full_mesh], show_skybox=False, bg_color=(1, 1, 1, 1))

# --- Save as colored mesh ---
o3d.io.write_triangle_mesh("output/deformed_wireframe_cylinders_colored.ply", full_mesh)
print("✅ Saved to output/deformed_wireframe_cylinders_colored.ply")


[Open3D INFO] Window window_0 created.
[Open3D INFO] EGL headless mode enabled.
[Open3D INFO] ICE servers: ["stun:stun.l.google.com:19302", "turn:user:password@34.69.27.100:3478", "turn:user:password@34.69.27.100:3478?transport=tcp"]
[Open3D INFO] Set WEBRTC_STUN_SERVER environment variable add a customized WebRTC STUN server.
[Open3D INFO] WebRTC Jupyter handshake mode enabled.


KeyboardInterrupt: 